In [3]:
with open("alice.txt") as file:
    text = file.read()

In [4]:
len(text)

145178

In [6]:
import torch
from torch import nn,optim
from torch.nn import functional as F
import numpy as np

In [8]:
chars = list(set(text))
indexer = {char: index for (index, char) in enumerate(chars)}

In [11]:
text_enc = [indexer[ch] for ch in text]

In [65]:
def toOneHot(batch):
    batch_flat = batch.flatten()
    oneHot_flat = np.zeros((len(batch_flat), len(chars)))
    oneHot_flat[range(len(batch_flat)), batch_flat] = 1
    oneHot = oneHot_flat.reshape(batch.shape[0], batch.shape[1], -1)
    return oneHot

In [44]:
batch_size = 100
text_seq = np.array(text_enc)[:(len(text)//100 * 100)].reshape(batch_size,-1)
text_seq

array([[32, 24,  3, ..., 33, 56, 39],
       [22, 65, 29, ..., 38, 19, 18],
       [ 3, 18, 33, ..., 64, 43, 18],
       ...,
       [48, 33, 51, ..., 29, 27, 18],
       [30, 22,  1, ..., 64, 33,  1],
       [38, 51, 33, ..., 51, 65, 65]])

In [97]:
class LSTM(nn.Module):
    def __init__(self, char_len, hidden_size, n_layers):
        super().__init__();
        self.hidden_size  = hidden_size
        self.n_layers = n_layers
        self.lstm  = nn.LSTM(char_len,hidden_size, n_layers, batch_first = True)
        self.output = nn.Linear(hidden_size, char_len)
    def forward(self, x, states):
        out, states = self.lstm(x, states)
        out = out.clone().contiguous().view(-1, self.hidden_size)
        out = self.output(out)
        return out,states
    def init_states(self, batch_size):
        hidden = next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_size).zero_()
        cell = next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_size).zero_()
        states = (hidden, cell)
        return states 

In [109]:
seq_len = 50
batch_size = 100
model = LSTM(len(chars), 256, 2).to("cuda")
epochs = 500
optimizer = optim.Adam(model.parameters(),  lr = 0.001)
loss_funct = nn.CrossEntropyLoss()

In [111]:
loss_arr = []
for e in range(1, epochs + 1):
    loss_cur = 0;
    states = model.init_states(batch_size)
    for b in range(0, text_seq.shape[1], seq_len):
        x_batch = text_seq[:, b:b+seq_len]
        if b == text_seq.shape[1]-seq_len:
            y_batch = text_seq[:, b+1:b+seq_len]
            y_batch = np.hstack((y_batch, indexer["."] *np.ones((y_batch.shape[0]))))
        elif b + 1 == text_seq.shape[1]:
            y_batch = indexer["."] * np.ones((y_batch.shape[0]))
        else:
            y_batch = text_seq[:, b+1:b+seq_len+1]
#             print(b)
    
        x = torch.Tensor(toOneHot(x_batch)).to("cuda")
        y = torch.Tensor(y_batch).view(-1).to("cuda")
        pred, states = model(x, states)
        states = tuple([each.data for each in states])
#         print(pred.shape, y.shape)
        loss = loss_funct(pred, y.long())
        
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()
        loss_cur+=loss.item()

    print(e, loss_cur)
    loss_arr.append(loss_cur)
    
    
        

1 96.19911479949951
2 96.15578126907349
3 96.02055621147156
4 95.8379979133606
5 95.26189231872559
6 92.93992853164673
7 87.88155674934387
8 83.18486881256104
9 79.66957521438599
10 77.1502947807312
11 75.15795707702637
12 73.49986672401428
13 72.27524590492249
14 70.94341778755188
15 69.75299406051636
16 68.52092337608337
17 67.50317811965942
18 66.29314184188843
19 65.30277609825134
20 64.2870991230011
21 63.34780144691467
22 62.453773856163025
23 61.57707703113556
24 60.79306495189667
25 59.98186707496643
26 59.162375926971436
27 58.384562730789185
28 57.68370747566223
29 56.980650424957275
30 56.24401533603668
31 55.517231464385986
32 54.851921796798706
33 54.22270846366882
34 53.555848360061646
35 52.998873233795166
36 52.28556740283966
37 51.64685344696045
38 51.013728618621826
39 50.44241213798523
40 49.914626121520996
41 49.3010174036026
42 48.58880913257599
43 48.14008319377899
44 47.636850357055664
45 46.90933549404144
46 46.36882996559143
47 45.905805230140686
48 45.63516521

365 0.19203219143673778
366 0.1846058696974069
367 0.17571164621040225
368 0.17314602667465806
369 0.16869495576247573
370 0.16164210729766637
371 0.16086007072590292
372 0.15897930227220058
373 0.1522150725359097
374 0.14543754479382187
375 0.14046132017392665
376 0.13668901240453124
377 0.13402553333435208
378 0.13006262050475925
379 0.1277496423572302
380 0.12526462553068995
381 0.12382270151283592
382 0.1204227446578443
383 0.12025246326811612
384 0.11741758068092167
385 0.1287961604539305
386 0.1282022978994064
387 0.12128876615315676
388 0.11472142988350242
389 0.11088114039739594
390 0.10877395886927843
391 0.11037003644742072
392 0.10745563334785402
393 0.10590660205343738
394 0.10237309767398983
395 0.1006964729167521
396 0.09838964138180017
397 0.09779697470366955
398 0.095458296360448
399 0.09473382856231183
400 0.09324422955978662
401 0.09324005816597492
402 0.09260698047000915
403 0.11495242430828512
404 0.1337339497404173
405 0.12736613542074338
406 0.11729642609134316
40

In [94]:
text_seq.shape

(100, 1451)

In [117]:
starter = "So she was considering in her own mind us on,"
states = None
model.eval()
model.to("cpu")
for ch in starter:
    x = np.array([[indexer[ch]]])
    x = toOneHot(x)
    x = torch.Tensor(x)
    
    pred, states = model(x, states)

counter = 0
while starter[-1] != "." and counter < 50:
    counter+=1
    x = np.array([[indexer[starter[-1]]]])
    x = toOneHot(x)
    x = torch.Tensor(x).to("cpu")
    
    pred,  states = model(x, states)
    pred = F.softmax(pred,  dim  = 1)
    p, top = pred.topk(10)
    p = p.detach().numpy()[0]
    top  = top.numpy()[0]
    index =  np.random.choice(top, p = p/p.sum())
    
    starter += chars[index]
    print(starter)

So she was considering in her own mind us on, 
So she was considering in her own mind us on, t
So she was considering in her own mind us on, th
So she was considering in her own mind us on, tho
So she was considering in her own mind us on, thou
So she was considering in her own mind us on, thoug
So she was considering in her own mind us on, though
So she was considering in her own mind us on, though 
So she was considering in her own mind us on, though t
So she was considering in her own mind us on, though th
So she was considering in her own mind us on, though the
So she was considering in her own mind us on, though the

So she was considering in her own mind us on, though the
L
So she was considering in her own mind us on, though the
Lo
So she was considering in her own mind us on, though the
Lor
So she was considering in her own mind us on, though the
Lory
So she was considering in her own mind us on, though the
Lory 
So she was considering in her own mind us on, though the
Lory h
S